In [ ]:
from google.colab import files


uploaded = files.upload()

In [ ]:
# @title
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pylab as plt
import sys
import json
import scipy
from scipy import signal
from numpy.linalg import inv
from numpy import sqrt
import random
import matplotlib.pylab as plt
from matplotlib import pyplot
import pandas as pd
import collections
from scipy import signal
from numpy.linalg import inv
np.set_printoptions(suppress=True)

## RLS algorithm


In [ ]:


def make_reg_matrix(serie: np.ndarray, p: int):
    """
    series : 1-D array of the time series
    p      : model order (number of regressors)
    Returns:
        X : array of shape [N-p, p]  (each row = [y(t-1), …, y(t-p)])
        d : array of shape [N-p]     (target values y(t))
    """

    N = len(serie)
    X = np.column_stack([serie[i:N-p+i] for i in range(p, 0, -1)])
    d = serie[p:]
    return X, d


# ------------------------------------------------------------------
# 2. RLS Filter
# ------------------------------------------------------------------
class FilterRLS_beta:
    def __init__(self):
        pass

    def run(self, d, X, passo, lamb, gama, ordem, alpha=None):
        """
        d       : target vector (shape [N])
        X       : design matrix (shape [N, order])
        passo   : prediction horizon (not used during training)
        lamb    : fixed λ (used when alpha is None)
        gama    : initial P setting (P₀ = γ·I)
        ordem   : number of regressors (columns of X)
        alpha   : enables adaptive λ when provided (typical range 0.002–0.01)
        """

        N = len(d)
        P = gama * np.identity(ordem)
        w = np.zeros(ordem)

        y_hat = np.zeros(N)
        e     = np.zeros(N)
        w_hist = np.zeros((N, ordem))


        sigma_hat = 1e-8
        beta_var  = 0.95

        for k in range(N):
            phi = X[k]

            y_hat[k] = np.dot(w, phi)
            e[k]     = d[k] - y_hat[k]

            # ---- λ  -------------------------------------------
            if alpha is not None:
                sigma_hat = beta_var * sigma_hat + (1 - beta_var) * e[k]**2
                lambda_k  = 1.0 - alpha * min(1.0, abs(e[k]) / np.sqrt(sigma_hat))
            else:
                lambda_k = lamb
            # --------------------------------------------------------------


            g_num = P @ phi
            g_den = lambda_k + phi.T @ g_num
            g     = g_num / g_den


            w = w + g * e[k]

            P = (P - np.outer(g, phi) @ P) / lambda_k

            w_hist[k] = w


        phi_future = X[-1].copy()
        pred = np.dot(w, phi_future)

        return y_hat, e, w_hist, pred




### The following code generates the plots

In [ ]:
# @title
def plot_predicao(anos, serie_real, predicao, erro, passo, valor_real_final, valor_pred_final):

    anos        = np.array(anos).astype(int)
    serie_real  = np.array(serie_real)
    predicao    = np.array(predicao)
    erro        = np.array(erro)

    max_len = len(anos) - passo
    k = min(len(predicao), len(erro), max_len, len(serie_real) - passo)

    anos_plot       = anos[passo:passo + k]
    serie_real_plot = serie_real[passo:passo + k]
    predicao_plot   = predicao[:k]
    erro_plot       = erro[:k]
    erro_rel_plot = erro_plot / serie_real_plot
    erro_rel_final = (valor_real_final - valor_pred_final) / valor_real_final

    if not (len(anos_plot) == len(serie_real_plot) == len(predicao_plot) == len(erro_plot)):
        print(f"Erro: tamanhos incompatíveis! "
              f"anos({len(anos_plot)}), real({len(serie_real_plot)}), "
              f"pred({len(predicao_plot)}), erro({len(erro_plot)})")
        return

    plt.figure(figsize=(10, 5))
    plt.plot(anos_plot, serie_real_plot, label="Valor Real",    marker='o', linestyle='-')
    plt.plot(anos_plot, predicao_plot,   label="Predição RLS", marker='s', linestyle='--')

    ultimo_ano = int(anos[-1])
    plt.scatter(ultimo_ano, valor_real_final,
                marker='o', s=100, color='blue',   label='Último Valor Real')
    plt.scatter(ultimo_ano, valor_pred_final,
                marker='*', s=150, color='orange', label='Predição Final')

    plt.xlabel("Ano"); plt.ylabel("Valor")
    plt.title("Real vs Predição (com Último Ano)")
    plt.legend(); plt.grid(True); plt.show()

    plt.figure(figsize=(10,5))
    plt.plot(
        anos_plot,
        erro_rel_plot,
        label="Erro Relativo",
        marker='d',
        linestyle='-',
        color='red'
    )

    plt.scatter(
        ultimo_ano,
        erro_rel_final,
        marker='*',
        s=150,
        color='red',
        label="Erro Relativo Final"
    )

    plt.axhline(0, linestyle='--', color='black')
    plt.xlabel("Ano")
    plt.ylabel("Erro Relativo")
    plt.title("Erro Relativo da Predição RLS")
    plt.legend()
    plt.grid(True)
    plt.show()

### Abaixo a função que roda a predição para cada DMU

In [ ]:


# @title
def aplicar_predicao(matriz_estados_anos, passo, anos):

  filt_paramet = 2
  lamb = 0.99
  gama = 0.1



  matriz_estados_anos.columns = matriz_estados_anos.columns.str.strip()


  numpy_arrays = {}

  resultados = []
  for index, row in matriz_estados_anos.iterrows():

    estado = row["Country Name"]
    serie_temporal_completa_cada_estado = row.iloc[1:].astype(float).values

    numpy_arrays[estado] = np.array(serie_temporal_completa_cada_estado)


    vetor_serie_temporal_cortado = serie_temporal_completa_cada_estado[:(-passo)]

    valor_do_ano_atual = serie_temporal_completa_cada_estado[-1-passo]

    valor_real_do_ano_da_predicao = serie_temporal_completa_cada_estado[-1]




    N = len(vetor_serie_temporal_cortado)
    x = [[0] * filt_paramet]
    aux = np.concatenate((np.zeros(filt_paramet), vetor_serie_temporal_cortado))
    for k in range(filt_paramet, len(vetor_serie_temporal_cortado) + filt_paramet):
        x.append([aux[k - i] for i in range(0, filt_paramet)])

    X, d_cortado = make_reg_matrix(vetor_serie_temporal_cortado, filt_paramet)

    Rf = FilterRLS_beta()
    Ry, Re, Rw, Rpredicao = Rf.run(
        d=d_cortado,
        X=X,
        passo=passo,
        lamb=0.99,
        gama=0.1,
        ordem=filt_paramet,
        alpha=0.005
        )


    resultados.append({
        "Estado": estado,
        "Valor Atual": valor_do_ano_atual,
        "Valor Real do ano de predicao": valor_real_do_ano_da_predicao,
        "Valor Predito": Rpredicao
    })

    plot_predicao(anos, vetor_serie_temporal_cortado, Ry, Re, passo, valor_real_do_ano_da_predicao, Rpredicao)

  return resultados



In [ ]:
# @title
arquivo_excel = pd.ExcelFile("Data.xlsx")

dados_abas = {nome: pd.read_excel(arquivo_excel, sheet_name=nome)
              for nome in arquivo_excel.sheet_names}

lista_de_abas = [pd.read_excel(arquivo_excel, sheet_name=nome)
                 for nome in arquivo_excel.sheet_names]

input1, input2, output1 = lista_de_abas


In [ ]:

matriz_estados_anos = output1
nome_arquivo= "output1"


vetor_passos = [1, 3, 5]




anos = matriz_estados_anos.columns[1:].astype(str).tolist()
print(f"Anos{anos}")

display(matriz_estados_anos.head())

for passo in vetor_passos:

  print(f"Passo {passo}")

  resultados = aplicar_predicao(matriz_estados_anos, passo, anos)

  df_resultados = pd.DataFrame(resultados)

  nome_arquivo_resultados =  f"Resultado_{nome_arquivo}_passo_{passo}.xlsx"

  df_resultados.to_excel(nome_arquivo_resultados, index=False)

  files.download(nome_arquivo_resultados)




In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plotar_series(df, titulo):
    df = df.set_index(df.columns[0])

    df_transposto = df.T

    plt.figure(figsize=(10,6))

    for pais in df_transposto.columns:
        plt.plot(df_transposto.index, df_transposto[pais], label=pais)

    plt.title(f'Série Temporal - {titulo}')
    plt.xlabel('Ano')
    plt.ylabel('Valor')
    plt.legend()
    plt.grid(True)
    plt.show()

plotar_series(input1, 'Input 1')
plotar_series(input2, 'Input 2')
plotar_series(output1, 'Output 1')


In [ ]:
def plotar_series_selecionadas(df, titulo, paises_interesse):
    df = df.set_index(df.columns[0])

    df_filtrado = df.loc[paises_interesse]

    df_transposto = df_filtrado.T

    plt.figure(figsize=(10,6))

    for pais in df_transposto.columns:
        plt.plot(df_transposto.index, df_transposto[pais], label=pais)

    plt.xticks(fontsize=11, rotation=45)
    plt.yticks(fontsize=11)
    plt.xlabel('Year', fontsize=20)
    plt.ylabel(f'{titulo}', fontsize=18)
    plt.legend(fontsize=15)
    plt.grid(True)
    plt.show()

paises_desejados = ['Slovak Republic', 'Austria']

plotar_series_selecionadas(input1, 'GDP per capita', paises_desejados)
plotar_series_selecionadas(input2, 'Expenditure on primary education', paises_desejados)
plotar_series_selecionadas(output1, 'School enrollment, primary', paises_desejados)
